In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [2]:
today = '2024-10-31'

print_to_excel = True
excel_tofolder_on_Z = True

In [4]:
base = 'RISKACCESS'
base = 'xxwh'
export_data = export_from_WHWEEK(f'''select * 
from {base}."unifiedOcp"
where "sourceTable" = 'sapBwInvestmentReport'
and "reportDate" = TO_DATE('{today}', 'YYYY-MM-DD')
--fetch first 10 rows only''')
export_data.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:176: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


37032

In [30]:
data_work = export_data.copy() # copy export data
data_work[['Завод','Вид_проекта','Версия_035','Версия_045','Версия_075','Версия_085']] = np.nan #add empty cols
# work with cols
data_work['Проект'] = data_work['recordId'].str.split('-').str[0]
data_work.maturityDateTo = data_work.maturityDateTo.astype(str).apply(lambda x: '-'.join([x[:-3],'01']))
data_work.maturityDateTo = pd.to_datetime(data_work.maturityDateTo)
data_work['Календарный_год'] = data_work['maturityDateTo'].dt.year
data_work['Календарный_месяц'] = data_work['maturityDateTo'].dt.month
data_work = data_work[['holding','ocpSegment','salesUnitsId','Завод','recordId','counterpartyCode','counterpartySegment','Вид_проекта','accountSubtype','Проект','accountCode','maturityDateTo','Календарный_год','Календарный_месяц','effectiveCurrency','Версия_035','Версия_045','Версия_075','Версия_085','volumeNotionalCurrency','usdRate','volumeUsd']]
data_work = data_work.rename(columns={'recordId':'СПП-элемент','accountSubtype':'Профиль_проекта','accountCode':'Вид_затрат'})

In [32]:
# Аня, в файлик по инвестициям надо добавить еще одну страницу, куда положить агрегированный набор данных по сокращенному списку полей. 
# Поля следующие. По расчётным полям amount поставить фильтр <>0
pivot_data = data_work[data_work.volumeNotionalCurrency!=0]
pivot_data = pivot_data.groupby(as_index=False, by=['holding','salesUnitsId','ocpSegment','effectiveCurrency','Календарный_год','Календарный_месяц','counterpartySegment','maturityDateTo'])[['volumeNotionalCurrency','volumeUsd']].agg(sum)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_23332\3280017617.py:4: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_data = pivot_data.groupby(as_index=False, by=['holding','salesUnitsId','ocpSegment','effectiveCurrency','Календарный_год','Календарный_месяц','counterpartySegment','maturityDateTo'])[['volumeNotionalCurrency','volumeUsd']].agg(sum)


In [34]:
### to excel
for holding in ['EUROCHEM', 'SUEK']:
    data = 0
    if print_to_excel == True:
        Output_file = f'{str(date.today())}_Invest_nv_{holding}.xlsx'
        if excel_tofolder_on_Z == True:
            Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'
            Output_file = Output_path + Output_file
        data = data_work[data_work.holding==holding]
        data.to_excel(Output_file, sheet_name='Invest', index=False)
        new_list(pivot_data[pivot_data.holding==holding], Output_file, sheet_name='pivot')